In [1]:
import os
from google import genai

# create client
api_key = os.getenv("GEMINI_API_KEY","xxx")
client = genai.Client(api_key=api_key)

In [ ]:
# Used to generate valid JSON Schema that can be used to generate structured output with Gemini
meta_prompt = """You are a JSON Schema expert. 
Your task is to create JSON schema based on the user input. 
The schema will be used for extra data.  

You must also make sure:
- All fields in an object are set as required
- All objects must have properties defined
- Order matters! If the values are dependent or would require additional information, 
make sure to include the additional information in the description. 
Same counts for "reasoning" or "thinking" should come before the conclusion.
- $defs must be defined under the schema param
- Return only the schema JSON not more, use ```json to start and ``` to end the JSON schema

Restrictions:
- You cannot use examples, if you think examples are helpful include them in the description.
- You cannot use default values, If you think default are helpful include them in the description.
- Never include a $schema
- The "type" needs to be a single value, no arrays

Guidelines:
- If the user prompt is short define a single object schema and fields based on your knowledge.
- If the user prompt is in detail about the data only use the data in the schema. 
Don't add more fields than the user asked for.

Examples:

Input: Cookie Recipes
Output: ```json
{{
   "title":"Cookie Recipe",
   "description":"Schema for a cookie recipe, including ingredients and quantities. 
   The 'ingredients' array lists each ingredient along with its corresponding quantity 
   and unit of measurement. The 'instructions' array provides a step-by-step guide to 
   preparing the cookies. The order of instructions is important.",
   "type":"object",
   "properties":{{
      "name":{{
         "type":"string",
         "description":"The name of the cookie recipe."
      }},
      "description":{{
         "type":"string",
         "description":"A short description of the cookie, including taste and textures."
      }},
      "ingredients":{{
         "type":"array",
         "description":"A list of ingredients required for the recipe.",
         "items":{{
            "type":"object",
            "description":"An ingredient with its quantity and unit.",
            "properties":{{
               "name":{{
                  "type":"string",
                  "description":"The name of the ingredient (e.g., flour, sugar, butter)."
               }},
               "quantity":{{
                  "type":"number",
                  "description":"The amount of the ingredient needed."
               }},
               "unit":{{
                  "type":"string",
                  "description":"The unit of measurement for the ingredient 
                  (e.g., cups, grams, teaspoons). Use abbreviations like 'tsp' for teaspoon and 
                  'tbsp' for tablespoon."
               }}
            }},
            "required":[
               "name",
               "quantity",
               "unit"
            ]
         }}
      }},
      "instructions":{{
         "type":"array",
         "description":"A sequence of steps to prepare the cookie recipe. 
         The order of instructions matters.",
         "items":{{
            "type":"string",
            "description":"A single instruction step."
         }}
      }}
   }},
   "required":[
      "name",
      "description",
      "ingredients",
      "instructions"
   ],
   "$defs":{{
      "ingredient":{{
         "type":"object",
         "description":"An ingredient with its quantity and unit.",
         "properties":{{
            "name":{{
               "type":"string",
               "description":"The name of the ingredient (e.g., flour, sugar, butter)."
            }},
            "quantity":{{
               "type":"number",
               "description":"The amount of the ingredient needed."
            }},
            "unit":{{
               "type":"string",
               "description":"The unit of measurement for the ingredient 
               (e.g., cups, grams, teaspoons). Use abbreviations like 'tsp' for teaspoon and 
               'tbsp' for tablespoon."
            }}
         }},
         "required":[
            "name",
            "quantity",
            "unit"
         ]
      }}
   }}
}}
```

Input: Book with title, author, and publication year.
Output: ```json
{{
    "type": "object",
    "properties": {{
        "title": {{
            "type": "string",
            "description": "The title of the book."
        }},
        "author": {{
            "type": "string",
            "description": "The author of the book."
        }},
        "publicationYear": {{
            "type": "integer",
            "description": "The year the book was published."
        }}
    }},
    "required": [
        "title",
        "author",
        "publicationYear"
    ],
}}
```

Input: {user_input}"""

In [9]:
from google import genai
import re 
import json
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=meta_prompt.format(user_input="Recipe for starting a manufacturing company"),
)
# Extract the JSON schema from the response
match = re.search(r"```json\s*(.*?)\s*```", response.text, re.DOTALL)
if match:
    json_schema = json.loads(match.group(1).strip())
else:
    json_schema = None
print(json.dumps(json_schema, indent=2))

{
  "title": "Manufacturing Company Startup Recipe",
  "description": "A detailed recipe for starting a manufacturing company, including steps from initial concept to production and sales. The order of steps is crucial for success.",
  "type": "object",
  "properties": {
    "conceptAndPlanning": {
      "type": "object",
      "description": "Phase focused on defining the product, market, and business strategy.",
      "properties": {
        "ideaValidation": {
          "type": "string",
          "description": "Detailed description of how the initial product idea was validated. Focus on research and customer feedback."
        },
        "marketAnalysis": {
          "type": "string",
          "description": "Comprehensive analysis of the target market, including size, trends, and competitive landscape."
        },
        "businessPlan": {
          "type": "string",
          "description": "Detailed business plan outlining the company's mission, vision, goals, and strategies f